In [ ]:
# ✅ STEP 1: Install packages
!pip install -q gradio tensorflow

# ✅ STEP 2: Imports
import gradio as gr
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# ✅ STEP 3: Load Fashion MNIST data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# ✅ STEP 4: Preprocess for CNN
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# ✅ STEP 5: Build CNN model
model = models.Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ✅ STEP 6: Train (quickly for demo)
model.fit(x_train, y_train, epochs=3, validation_split=0.1)

# ✅ STEP 7: Prediction function
def predict_image(img):
    img = img.convert('L').resize((28, 28))  # Grayscale + Resize
    img_array = np.array(img).astype('float32') / 255.0
    img_array = img_array.reshape(1, 28, 28, 1)
    prediction = model.predict(img_array)
    return {class_names[i]: float(prediction[0][i]) for i in range(10)}

# ✅ STEP 8: Gradio Interface
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="🧠 Fashion MNIST Image Classifier",
    description="Upload a 28x28 grayscale image to predict the fashion item."
)

# ✅ STEP 9: Launch
interface.launch()
